## 1. Git Clone the repo

In [ ]:
import os
from google.colab import drive

# 1. Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

%cd /content

# 2. Handle Repository (Clone or Update)
repo_dir = "/content/SDXL_GGUF_Quantize_Tool"
repo_url = "https://github.com/magekinnarus/SDXL_GGUF_Quantize_Tool.git"

if not os.path.exists(repo_dir):
    print("Cloning repository...")
    !git clone {repo_url} {repo_dir}
else:
    print("Repository already exists. Pulling latest updates...")
    %cd {repo_dir}
    !git pull

# 3. Finalize directory location
%cd {repo_dir}

### 1.1 Create .env (Optional) or you can upload yours to /content folder

In [ ]:
%%writefile /content/.env
HUGGINGFACE_TOKEN=##your_huggingface_token##
CIVITAI_TOKEN=##your_civitai_token##

## 2. Download models from CivitAI. I am using a model catalogue json file. You can modify and upload your own json file.

### 2.1 Inatall Aria2 and download the model catalogue json

In [ ]:
!apt -y install -qq aria2

# Ensure dotenv is installed
try:
    import dotenv
except ImportError:
    print("Installing python-dotenv...")
    !pip install python-dotenv

import shutil
import subprocess
import os
from dotenv import load_dotenv
from IPython.display import clear_output

# Define the path to your .env file (you can adjust this if your .env file is located elsewhere)
token_path = "/content"

# Load environment variables from the .env file
# This will look for a .env file in the current working directory, or you can specify the path
load_dotenv(dotenv_path=os.path.join(token_path, ".env"))

# Get the Hugging Face token from environment variables
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

# Load models_configs file
!wget -P /content/models https://huggingface.co/Old-Fisherman/SDXL_Models/resolve/main/civit_models_config.json

#Clear log printout
clear_output(wait=False)
print("✅ all processes completed.")

### 2.2 Download from CivitAI

In [ ]:
import os
import json
import subprocess
from dotenv import load_dotenv
from IPython.display import clear_output

# 1. Load Model Configurations from both Public and Private files
model_path = "/content/models"
config_files = ['civit_models_config.json', 'civit_models_P1_config.json']
ALL_MODELS = []

for file_name in config_files:
    full_path = os.path.join(model_path, file_name)
    if os.path.exists(full_path):
        with open(full_path, 'r') as f:
            data = json.load(f)
            # Extract models from Categories (Checkpoints/LoRA) and Sub-types (SDXL/Pony/etc)
            for category in data.values():
                for model_list in category.values():
                    ALL_MODELS.extend(model_list)
        print(f"✅ Loaded: {file_name}")
    else:
        print(f"⚠️ Warning: {file_name} not found.")

# 2. Create Model Directories based on the 'dir' keys in the JSON
# This automatically handles 'checkpoints', 'loras', and 'unet' folders
unique_dirs = set(m['dir'] for m in ALL_MODELS)
for d in unique_dirs:
    os.makedirs(os.path.join(model_path, d), exist_ok=True)
print(f"Directories verified: {', '.join(unique_dirs)}")

# 3. Define the download script
def download_model(selector):
    # Find the model by 'nr' (int) or 'alias' (str)
    config = next((m for m in ALL_MODELS if m['nr'] == selector or m['alias'] == selector), None)

    if not config:
        print(f"❌ Configuration not found for: {selector}")
        return

    print(f"\n🚀 Starting download: {config['name']} ({config['alias']})")

    base_url = "https://civitai.com/api/download/models/"
    # Ensure environment variables are loaded for the token
    load_dotenv(os.path.join("/content", ".env"))
    CIVITAI_TOKEN = os.getenv('CIVITAI_TOKEN', '')

    url = f"{base_url}{config['id']}?token={CIVITAI_TOKEN}"
    download_dir = os.path.join(model_path, config['dir'])

    try:
        command = [
            'aria2c', '--console-log-level=warn', '-c', '-x', '16', '-s', '16', '-k', '1M',
            '--dir', download_dir, '--out', config['name'], url
        ]
        subprocess.check_call(command)
        print(f"✅ Download completed: {config['name']}")
    except subprocess.CalledProcessError as e:
        print(f"🛑 Error downloading {config['name']}: {e}")

# 4. Select Models (Use Number or Alias)
SELECTED_MODELS = [
    "innovision", "realcartoon_xl", "juggernaut", "blendermix"
]

print("Starting download process for selected models...")
for selector in SELECTED_MODELS:
    download_model(selector)

# Clear log printout
clear_output(wait=False)
print("✅ All selected model downloads completed successfully.")

## 3. Run the app!

In [ ]:
# --- RUNNING THE UI ---
!pip install -q -r requirements-gradio.txt

import os
import gguf
import app_gradio

# 1. Ensure we are in the repo directory
%cd /content/SDXL_GGUF_Quantize_Tool

# 3. Launch strictly within the Colab iframe
print("\n--- Launching SDXL GGUF Quantizer (Local Only) ---")

app_gradio.demo.launch(
    share=True,      # No public link generated
    inline=True,      # Displays inside this Colab cell
    debug=True,       # Helpful for tracking quantization progress/errors
    height=800
)